In [ ]:
import cv2
import numpy as np
import os
import logging
from pathlib import Path

In [ ]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Directories
input_dir = Path("./satellite_imagery")
output_dir = Path("./satellite_imagery_cropped")
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Constants
PADDING = 5
lower_blue = np.array([0,  25, 235])
upper_blue = np.array([0,  45, 255])

In [ ]:
def process_image_pair(base_path: Path, boxed_path: Path):
    logging.debug(f"Processing pair:\n  Base: {base_path.name}\n  Boxed: {boxed_path.name}")

    # Load images
    base_img = cv2.imread(str(base_path), 1)
    boxed_img = cv2.imread(str(boxed_path), 1)

    # Channel conversion (RGB --> BGR)
    base_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)
    boxed_img = cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB)

    if base_img is None or boxed_img is None:
        logging.warning(f"Could not load image pair: {base_path.name}")
        return
      
    # Create mask where boxed image is exactly blue
    mask = cv2.inRange(boxed_img, lower_blue, upper_blue)

    # Get contours of the shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        logging.warning(f"No blue shape found in {boxed_path.name}")
        return

    contour = max(contours, key=cv2.contourArea)

    # Create a filled mask from the contour
    mask_filled = np.zeros_like(mask)
    cv2.drawContours(mask_filled, [contour], -1, 255, thickness=cv2.FILLED)

    # Create white background image
    white_background = np.ones_like(base_img, dtype=np.uint8) * 255

    # Copy pixels from base image where mask is non-zero
    result_img = np.where(mask_filled[:, :, np.newaxis] == 255, base_img, white_background)

    # Crop to bounding box of the mask, with padding
    x, y, w, h = cv2.boundingRect(contour)
    x1 = max(x - PADDING, 0)
    y1 = max(y - PADDING, 0)
    x2 = min(x + w + PADDING, base_img.shape[1])
    y2 = min(y + h + PADDING, base_img.shape[0])

    result_cropped = result_img[y1:y2, x1:x2]

    # Save output
    output_name = base_path.name.replace('_base.png', '_cropped.png')
    output_path = output_dir / output_name
    cropped = cv2.cvtColor(result_cropped, cv2.COLOR_BGR2RGB)
    cv2.imwrite(str(output_path), cropped)
    
    logging.info(f"Cropped image saved: {output_path.name}")

In [ ]:
def main():
    logging.info("Starting batch crop process...")

    boxed_files = sorted(input_dir.glob("*_boxed.png"))

    for boxed_path in boxed_files:
        base_path = Path(str(boxed_path).replace('_boxed.png', '_base.png'))
        if not base_path.exists():
            logging.warning(f"Base image not found for: {boxed_path.name}")
            continue

        try:
            process_image_pair(base_path, boxed_path)
        except Exception as e:
            logging.error(f"Error processing {boxed_path.name}: {e}")

    logging.info("Processing complete.")


In [ ]:
if __name__ == "__main__":
    main()